# ***תרגיל 6 של הפרוייקט***

**Setup & Install (Colab)**

In [ ]:
# ============================================================
# Block 1 — Setup & Install (Colab)
# ============================================================
!pip -q install transformers datasets accelerate scikit-learn

import os
import random
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

from sklearn.metrics import classification_report, confusion_matrix

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)


ImportError: cannot import name 'AdamW' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

**Reproducibility (Seeds)**

In [ ]:
# ============================================================
# Block 2 — Reproducibility (Seeds)
# ============================================================
def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)


**Config (Paths, Labels, Hyperparams)**

In [ ]:
# ============================================================
# Block 3 — Config (Paths, Labels, Data Rules)
# ============================================================
CSV_PATH = "/content/train-filtered_question_level.csv"  # change if needed

label2id = {"easy": 0, "medium": 1, "hard": 2}
id2label = {v: k for k, v in label2id.items()}
NUM_LABELS = 3

# Based on your analysis: 95th percentile = 44 tokens
MAX_LEN = 44

# Data rules you decided
SAMPLES_PER_CLASS = 7000

# Split ratios (exact counts will be enforced per class)
TRAIN_RATIO = 0.70
VAL_RATIO   = 0.15
TEST_RATIO  = 0.15


**Load Dataset (CSV) + Minimal Cleaning**

In [ ]:
# ============================================================
# Block 4 — Load Dataset + Minimal Cleaning (NO manual lowercasing)
# ============================================================
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"CSV not found at: {CSV_PATH}")

df = pd.read_csv(CSV_PATH)

required_cols = {"question", "level"}
if not required_cols.issubset(set(df.columns)):
    raise ValueError(f"CSV must contain columns: {required_cols}. Found: {df.columns}")

# Keep preprocessing minimal for Transformers
df["question"] = df["question"].astype(str)
df["level"] = df["level"].astype(str)

# Remove duplicates by question
df = df.drop_duplicates(subset=["question"]).reset_index(drop=True)

# Keep only valid labels
df = df[df["level"].isin(label2id.keys())].reset_index(drop=True)

print("Raw dataset size:", len(df))
print(df["level"].value_counts())


In [ ]:
# ============================================================
# Block 4.5 — Balance Dataset (Exactly 7000 per label) + Shuffle
# ============================================================
counts = df["level"].value_counts()
missing = [lbl for lbl in label2id.keys() if counts.get(lbl, 0) < SAMPLES_PER_CLASS]
if missing:
    raise ValueError(
        f"Not enough samples for labels: {missing}. "
        f"Counts: {counts.to_dict()}"
    )

balanced_df = (
    df.groupby("level", group_keys=False)
      .apply(lambda g: g.sample(n=SAMPLES_PER_CLASS, random_state=42))
      .reset_index(drop=True)
)

# Full shuffle after balancing
df = balanced_df.sample(frac=1.0, random_state=42).reset_index(drop=True)

print("Balanced dataset size:", len(df))
print(df["level"].value_counts())


**Stratified Split (Train / Val / Test)**

In [ ]:
# ============================================================
# Block 5 — Exact Balanced Split (Equal per label in Train/Val/Test)
# ============================================================
train_per_class = int(SAMPLES_PER_CLASS * TRAIN_RATIO)  # 4900
val_per_class   = int(SAMPLES_PER_CLASS * VAL_RATIO)    # 1050
test_per_class  = SAMPLES_PER_CLASS - train_per_class - val_per_class  # 1050

assert train_per_class + val_per_class + test_per_class == SAMPLES_PER_CLASS

def split_one_class(g: pd.DataFrame, seed: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    g = g.sample(frac=1.0, random_state=seed).reset_index(drop=True)
    train_g = g.iloc[:train_per_class]
    val_g   = g.iloc[train_per_class:train_per_class + val_per_class]
    test_g  = g.iloc[train_per_class + val_per_class:train_per_class + val_per_class + test_per_class]
    return train_g, val_g, test_g

train_parts, val_parts, test_parts = [], [], []

for label in label2id.keys():
    g = df[df["level"] == label].reset_index(drop=True)
    if len(g) != SAMPLES_PER_CLASS:
        raise ValueError(f"Expected {SAMPLES_PER_CLASS} samples for label={label}, got {len(g)}")

    tr_g, va_g, te_g = split_one_class(g, seed=42)
    train_parts.append(tr_g)
    val_parts.append(va_g)
    test_parts.append(te_g)

train_df = pd.concat(train_parts).sample(frac=1.0, random_state=42).reset_index(drop=True)
val_df   = pd.concat(val_parts).sample(frac=1.0, random_state=42).reset_index(drop=True)
test_df  = pd.concat(test_parts).sample(frac=1.0, random_state=42).reset_index(drop=True)

print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))
print("Train distribution:\n", train_df["level"].value_counts())
print("Val distribution:\n", val_df["level"].value_counts())
print("Test distribution:\n", test_df["level"].value_counts())


**PyTorch Dataset (Tokenizer-based)**

In [ ]:
# ============================================================
# Block 6 — PyTorch Dataset (Tokenizer-based; NO manual vocab)
# ============================================================
class TriviaDataset(torch.utils.data.Dataset):
    """
    Tokenizes each question using the EXACT tokenizer of the chosen Transformer model.
    Returns input_ids, attention_mask, and labels.
    """
    def __init__(self, dataframe: pd.DataFrame, tokenizer, max_len: int):
        self.df = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        text = self.df.loc[idx, "question"]
        label_str = self.df.loc[idx, "level"]
        label_id = label2id[label_str]

        encoded = self.tokenizer(
            text,
            padding="max_length",     # explicit padding
            truncation=True,          # explicit truncation
            max_length=self.max_len,
            return_tensors="pt"
        )

        item = {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "labels": torch.tensor(label_id, dtype=torch.long)
        }

        # BERT models return token_type_ids (segment ids)
        if "token_type_ids" in encoded:
            item["token_type_ids"] = encoded["token_type_ids"].squeeze(0)

        return item


**Training Utilities (Accuracy, Loops, Plots)**

In [ ]:
# ============================================================
# Block 7 — Training Utilities (Loss/Acc history + plots)
# ============================================================
def batch_accuracy(logits: torch.Tensor, labels: torch.Tensor) -> float:
    preds = torch.argmax(logits, dim=1)
    return (preds == labels).float().mean().item()

@dataclass
class TrainHistory:
    train_loss: List[float]
    val_loss: List[float]
    train_acc: List[float]
    val_acc: List[float]

def train_one_epoch(model, loader, optimizer, scheduler) -> Tuple[float, float]:
    model.train()
    total_loss, total_acc, n_batches = 0.0, 0.0, 0

    for batch in loader:
        batch = {k: v.to(DEVICE) for k, v in batch.items()}

        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        total_acc += batch_accuracy(logits, batch["labels"])
        n_batches += 1

    return total_loss / n_batches, total_acc / n_batches

@torch.no_grad()
def eval_one_epoch(model, loader) -> Tuple[float, float]:
    model.eval()
    total_loss, total_acc, n_batches = 0.0, 0.0, 0

    for batch in loader:
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()
        total_acc += batch_accuracy(logits, batch["labels"])
        n_batches += 1

    return total_loss / n_batches, total_acc / n_batches

def plot_history(history: TrainHistory, title: str) -> None:
    epochs = range(1, len(history.train_loss) + 1)

    plt.figure()
    plt.plot(epochs, history.train_loss, label="train_loss")
    plt.plot(epochs, history.val_loss, label="val_loss")
    plt.title(title + " — Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(epochs, history.train_acc, label="train_acc")
    plt.plot(epochs, history.val_acc, label="val_acc")
    plt.title(title + " — Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()


**Evaluation Utilities (Report + Confusion Matrix)**

In [ ]:
# ============================================================
# Block 8 — Evaluation Utilities (Report + Confusion Matrix)
# ============================================================
@torch.no_grad()
def predict(model, loader) -> Tuple[List[int], List[int]]:
    model.eval()
    all_preds, all_labels = [], []

    for batch in loader:
        labels_cpu = batch["labels"].numpy().tolist()
        batch = {k: v.to(DEVICE) for k, v in batch.items()}

        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy().tolist()

        all_preds.extend(preds)
        all_labels.extend(labels_cpu)

    return all_preds, all_labels

def show_metrics(y_true: List[int], y_pred: List[int], title: str) -> None:
    print("\n" + "="*70)
    print(title)
    print("="*70)

    print("\nClassification Report:")
    print(classification_report(
        y_true, y_pred,
        target_names=[id2label[i] for i in range(NUM_LABELS)],
        digits=4
    ))

    cm = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix (rows=true, cols=pred):")
    print(cm)


**One Experiment Runner (Model + Tokenizer + Fine-Tuning)**

In [ ]:
# ============================================================
# Block 9 — Experiment Runner (model + tokenizer + full fine-tuning)
# ============================================================
def run_experiment(
    model_name: str,
    run_name: str,
    lr: float,
    batch_size: int,
    epochs: int,
    optimizer_name: str = "adamw",
    weight_decay: float = 0.01,
    warmup_ratio: float = 0.10
) -> Dict:
    """
    Runs one full fine-tuning experiment and returns metrics + trained model.
    Designed to make hyperparameter experiments easy.
    """
    print("\n" + "#"*90)
    print(f"Run: {run_name}")
    print(f"Model: {model_name}")
    print(f"Params: lr={lr}, batch={batch_size}, epochs={epochs}, optimizer={optimizer_name}, MAX_LEN={MAX_LEN}")
    print("#"*90)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    train_ds = TriviaDataset(train_df, tokenizer, MAX_LEN)
    val_ds   = TriviaDataset(val_df, tokenizer, MAX_LEN)
    test_ds  = TriviaDataset(test_df, tokenizer, MAX_LEN)

    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader   = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    test_loader  = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=NUM_LABELS,
        label2id=label2id,
        id2label=id2label
    ).to(DEVICE)

    # Optimizer choice (as requested by lecturer: Adam vs AdamW)
    opt = optimizer_name.lower()
    if opt == "adamw":
        optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif opt == "adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    else:
        raise ValueError("optimizer_name must be 'adamw' or 'adam'")

    total_steps = len(train_loader) * epochs
    warmup_steps = int(total_steps * warmup_ratio)

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    history = TrainHistory(train_loss=[], val_loss=[], train_acc=[], val_acc=[])

    for ep in range(1, epochs + 1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, scheduler)
        va_loss, va_acc = eval_one_epoch(model, val_loader)

        history.train_loss.append(tr_loss)
        history.train_acc.append(tr_acc)
        history.val_loss.append(va_loss)
        history.val_acc.append(va_acc)

        print(f"Epoch {ep}/{epochs} | train_loss={tr_loss:.4f} train_acc={tr_acc:.4f} | val_loss={va_loss:.4f} val_acc={va_acc:.4f}")

    plot_history(history, title=run_name)

    # Test evaluation
    y_pred, y_true = predict(model, test_loader)
    show_metrics(y_true, y_pred, title=f"{run_name} — Test Evaluation")

    return {
        "run_name": run_name,
        "model_name": model_name,
        "lr": lr,
        "batch_size": batch_size,
        "epochs": epochs,
        "optimizer": optimizer_name,
        "history": history,
        "y_true": y_true,
        "y_pred": y_pred,
        "model": model,
        "tokenizer": tokenizer
    }


In [ ]:
# ============================================================
# Block 10 — Experiment Manager (Add ONE block per experiment)
# ============================================================
SMALL_MODEL = "bert-base-cased"
LARGE_MODEL = "bert-large-cased"

# Each experiment is ONE dict.
# To add a new experiment, you only add ONE new dict line.
EXPERIMENTS = [
    # ---- Small model (BERT base) runs ----
    {"run_name": "E1_BERT_BASE_adamw_lr3e-5_bs16_ep3", "model": SMALL_MODEL, "lr": 3e-5, "bs": 16, "ep": 3, "opt": "adamw"},
]

results = []
for exp in EXPERIMENTS:
    out = run_experiment(
        model_name=exp["model"],
        run_name=exp["run_name"],
        lr=exp["lr"],
        batch_size=exp["bs"],
        epochs=exp["ep"],
        optimizer_name=exp["opt"]
    )
    results.append(out)


דוגמאותתתתתתתתתתתתתתתתתתתתת

In [ ]:
# ============================================================
# Block 11 — Add More Experiments (Copy/Paste ONE dict per run)
# ============================================================
# Example experiments requested by lecturer:
# - change LR
# - change batch size
# - change epochs
# - change optimizer

EXPERIMENTS_MORE = [
    # LR experiments (keep everything else same)
    {"run_name": "E2_BERT_BASE_adamw_lr2e-5_bs16_ep3", "model": SMALL_MODEL, "lr": 2e-5, "bs": 16, "ep": 3, "opt": "adamw"},
    {"run_name": "E3_BERT_BASE_adamw_lr5e-5_bs16_ep3", "model": SMALL_MODEL, "lr": 5e-5, "bs": 16, "ep": 3, "opt": "adamw"},

    # Batch size experiment (if GPU allows)
    {"run_name": "E4_BERT_BASE_adamw_lr3e-5_bs32_ep3", "model": SMALL_MODEL, "lr": 3e-5, "bs": 32, "ep": 3, "opt": "adamw"},

    # Epochs experiments
    {"run_name": "E5_BERT_BASE_adamw_lr3e-5_bs16_ep2", "model": SMALL_MODEL, "lr": 3e-5, "bs": 16, "ep": 2, "opt": "adamw"},
    {"run_name": "E6_BERT_BASE_adamw_lr3e-5_bs16_ep4", "model": SMALL_MODEL, "lr": 3e-5, "bs": 16, "ep": 4, "opt": "adamw"},

    # Optimizer comparison
    {"run_name": "E7_BERT_BASE_adam_lr3e-5_bs16_ep3",  "model": SMALL_MODEL, "lr": 3e-5, "bs": 16, "ep": 3, "opt": "adam"},
]

for exp in EXPERIMENTS_MORE:
    out = run_experiment(
        model_name=exp["model"],
        run_name=exp["run_name"],
        lr=exp["lr"],
        batch_size=exp["bs"],
        epochs=exp["ep"],
        optimizer_name=exp["opt"]
    )
    results.append(out)


Large Model Run

In [ ]:
# ============================================================
# Block 12 — Large Model Run (Use the best config from BERT-base)
# ============================================================
# After you decide what is "best" from BERT-base runs, run BERT-large once or twice.
# Start conservative: smaller batch and slightly lower LR for stability.

EXPERIMENT_LARGE = [
    {"run_name": "E8_BERT_LARGE_adamw_lr2e-5_bs8_ep3", "model": LARGE_MODEL, "lr": 2e-5, "bs": 8, "ep": 3, "opt": "adamw"},
    # Optional: an even safer LR if needed
    # {"run_name": "E9_BERT_LARGE_adamw_lr1e-5_bs8_ep3", "model": LARGE_MODEL, "lr": 1e-5, "bs": 8, "ep": 3, "opt": "adamw"},
]

for exp in EXPERIMENT_LARGE:
    out = run_experiment(
        model_name=exp["model"],
        run_name=exp["run_name"],
        lr=exp["lr"],
        batch_size=exp["bs"],
        epochs=exp["ep"],
        optimizer_name=exp["opt"]
    )
    results.append(out)


Results Table

In [ ]:
# ============================================================
# Block 13 — (Optional) Results Table (Quick comparison)
# ============================================================
from sklearn.metrics import accuracy_score

summary_rows = []
for r in results:
    acc = accuracy_score(r["y_true"], r["y_pred"])
    summary_rows.append({
        "run_name": r["run_name"],
        "model": r["model_name"],
        "lr": r["lr"],
        "batch": r["batch_size"],
        "epochs": r["epochs"],
        "optimizer": r["optimizer"],
        "test_acc": acc
    })

summary_df = pd.DataFrame(summary_rows).sort_values(by="test_acc", ascending=False)
summary_df
